<a href="https://colab.research.google.com/github/joehawkens/MachineLearning/blob/main/FINAL_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXOPLANET PREDICTION MODEL
**GOAL: To predict exoplanet habitability using data collected from the Kepler Space Observatory sattelite.**

- Source: https://www.kaggle.com/datasets/nasa/kepler-exoplanet-search-results
- Data: https://raw.githubusercontent.com/joehawkens/MachineLearning/main/exoplanets.csv

## Data Cleaning